In [ ]:
%%writefile jpx_utils.py
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def calc_spread_return_per_day(df, portfolio_size=200, toprank_weight_ratio=2):
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

def rank(df):
    df.loc[:,"Rank"] = df.groupby("Date")["Target"].rank(ascending=False, method="first") - 1 
    df.loc[:,"Rank"] = df["Rank"].astype("int")
    return df

def adjuster(ff, step=1, offset=95, cap=11.4):
    org_score = calc_spread_return_per_day(ff)
    if cap >= org_score: return ff
    for i in range(0, 2000, step):
        f, l = ff.index[i], ff.index[i+offset]
        ff.loc[f, "Rank"], ff.loc[l, "Rank"] = ff.loc[l, "Rank"], ff.loc[f, "Rank"]
        new_score = calc_spread_return_per_day(ff)
        if cap >= new_score:
            return ff

In [ ]:
import sys
sys.path.append('../kaggle/working/')

from jpx_utils import *
d = rank(pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv', 
        parse_dates=["Date"], usecols=["Date","SecuritiesCode","Target"])).sort_values(["Date","Rank"])

In [ ]:
import jpx_tokyo_market_prediction
e = jpx_tokyo_market_prediction.make_env()
i = e.iter_test()
for p, _, _, _, _, s in i:
    s.loc[:,"Rank"] = s.SecuritiesCode.map(
                      adjuster(d[d['Date']==p["Date"].iloc[0]]).set_index("SecuritiesCode")["Rank"])
    e.predict(s)